<a href="https://colab.research.google.com/github/compartia/nlp_tools/blob/tensorflow-model/FuzzyMatcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
GLOBALS__={}

## MAIN, init, load code

In [0]:
#@title  { form-width: "300px", display-mode: "form" }
import os






# ''' AZ:-IMPORT CODE GITHUB----------------------------------------------😺------ '''
import sys
# ====================================
# ====================================
_git_branch = "tensorflow-model"  # @param {type:"string"}
# ====================================
# ====================================


def _init_import_code_from_gh():
  if 'GLOBALS__' not in globals():
    print('adding global GLOBALS__')
    global GLOBALS__
    GLOBALS__ = {}

  if '_init_import_code_from_gh' in GLOBALS__:
    print('👌 code already imported from GitHub!')
    return

  import subprocess
  def exec(x):
    r = subprocess.check_output(x, shell=True)
    r = r.decode('unicode-escape').encode('latin1').decode('utf8')
    print(r)

  print(f"fetching code from GitHub.....{_git_branch}")
  try:
    exec('rm -r nlp_tools')
  except:
    pass
  exec(f'git clone --single-branch --branch {_git_branch} https://github.com/compartia/nlp_tools.git nlp_tools')

  print('🦊 GIT revision:')
  exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

  sys.path.insert(0, 'nlp_tools')

  # self-test
  from text_tools import untokenize
  print(untokenize(['code', 'imported', 'OK 👍']))

  print('installing antiword...')
  exec('sudo apt-get install antiword')

  print('installing docx2txt...')
  exec("pip install docx2txt")

  GLOBALS__['_init_import_code_from_gh'] = True

  ''' AZ:-------------------------------------------------IMPORT CODE GITHUB-😺---'''
  print('❤️ DONE importing Code fro GitHub')


# AZ:-INIT ELMO-----------------------------------------------------------------------------------

import tensorflow_hub as hub

#
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML


 

# AZ:- THE CODE----------------------------------------------------------------------------------

def _init_the_code(reset=False):
  if '_init_the_code' in GLOBALS__ and not reset:
    print('👌 Code is alredy imported!')
    return

  from renderer import HtmlRenderer
  from renderer import to_multicolor_text
  from renderer import known_subjects_dict

  from structures import ContractSubject
  from contract_parser import ContractDocument3

  from ml_tools import ProbableValue

  from legal_docs import LegalDocument
  from renderer import as_warning, as_headline_3, as_offset, as_smaller

  class DemoRenderer(HtmlRenderer):

    def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
      html = self.to_color_text(tokens, weights, colormap, print_debug, _range)
      display(HTML(html))

    def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
      return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range)

    def render_multicolor_text(self, tokens, vectors, colormap, min_color=None, _slice=None):
      display(HTML(to_multicolor_text(tokens, vectors, colormap, min_color=min_color, _slice=_slice)))

     
     

     
  GLOBALS__['renderer'] = DemoRenderer()

  GLOBALS__['_init_the_code'] = True
  print("❤️ DONE initializing the code")

  # AZ:-------------------------------------------------Init Protocols context===


def read_doc(fn):
  import docx2txt, sys, os

  text = ''
  try:
    text = docx2txt.process(fn)

  except:
    print("Unexpected error:", sys.exc_info())
    os.system('antiword -w 0 "' + fn + '" > "' + fn + '.txt"')
    with open(fn + '.txt') as f:
      text = f.read()

  return text

def interactive_upload(filetype):
  from google.colab import files
  import docx2txt

  print(f'Please select "{filetype}" .docx file:')
  uploaded = files.upload()
  docs = []
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

    with open(fn, "wb") as df:
      df.write(uploaded[fn])
      df.close()

    # extract text

    text = ''
    try:
      text = docx2txt.process(fn)
    except:
      print("Unexpected error:", sys.exc_info())
      os.system('antiword -w 0 "' + fn + '" > "' + fn + '.txt"')
      with open(fn + '.txt') as f:
        text = f.read()
    print("Символов в оригинальном документе:", len(text))
    docs.append(text)
    return docs

  
# AZ:- ENDO OF THE THE CODE------------------------------------------------XXXX


# 1.
_init_import_code_from_gh()
 
# 3.
_init_the_code(True)



### the model 💎 for finding closest embedding to a pattern
1. At first stage, we calculate attention vectors (AV) for every pattern
2. Then, for each AV we're finding the closest point (`best point`) in text embedding space, 
3. Then we're calculating "improved" vectors -- we calculate distance from the `"best point"` (actually, from the best *window*)  to each point in text embedding space.  
  
WARNING: this should be used only if `best point` is close enough to the pattern

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
from tf_model import PatternSearchModel
PM = PatternSearchModel(tf, hub)


## Playground

In [0]:
 
want_upload = False #@param {type:"boolean"}
search_for = "\u0432 \u043B\u0438\u0446\u0435 \u0433\u0435\u043D\u0435\u0440\u0430\u043B\u044C\u043D\u043E\u0433\u043E \u0434\u0438\u0440\u0435\u043A\u0442\u043E\u0440\u0430" #@param {type:"string"}
from patterns import AbstractPatternFactory, FuzzyPattern

from text_tools import tokenize_text
import re
from text_normalize import normalize_text, replacements_regex
uploaded = """
Берлинская дада-ярмарка и Международная выставка сюрреализма в париж­ской галерее «Изящные искусства» в 1938 году стали высшими точками развития двух движений и подвели им итог. На «Сюрреалис­тической улице», за манекенами, выстроившимися в проходе в главный зал, располагались плакаты, приглашения, объявления и фотографии, отсылающие к ранним этапам сюрреализма. В главном зале, за оформление которого отвечал Марсель Дюшан 
 
, а за освеще­ние — Ман Рэй 
 
, картины 1920-х годов висели рядом с более ранними работами, что подчеркивало развити
е сюрреалистического «интернаци­она­ла». Зародившись как литературное течение, к концу 1930-х годов сюрреализм уже около 15 лет господствовал в художественном авангарде Парижа. Прежде чем пойти на спад с началом
Второй мировой войны, он стал частью светской культуры Парижа и даже до некотоРой
степени присягнул высокой моде, подобно тому как русский авангард — пусть совсем иначе — присягну 
в свое время революции. Изящество стиля, свойственное сюррелизму, способствоало этому 

­ сближению, которое, в свою очередь, упрочило положение многих представителей направления в обществе. 

Однако поначалу для литераторов и художников-бунтарей, ничуть не стремившихся к социаль­ному успеху, 
была куда более естественной связь с дадаизмом"""

#UNCOMMENT TO UPLOAD============================================================

 
if want_upload:
  uploaded = interactive_upload('Protocol')[0]
# 
_regex_addon = [
    (re.compile(r'[­]'), '-'),
]
TOKENS=tokenize_text( normalize_text(uploaded, replacements_regex+_regex_addon))



class PF(AbstractPatternFactory):
  def __init__(self):
    AbstractPatternFactory.__init__(self, None)
    self._build_ner_patterns()

  def _build_ner_patterns(self):
    def cp(name, tuples):
      return self.create_pattern(name, tuples)
   
    cp('_custom', ( '', search_for, ''))
    

# ---
pf = PF()

av = PM.find_patterns(text_tokens=TOKENS, patterns=pf.patterns)


 
fig = plt.figure(figsize=(20, 6))
ax = plt.axes()
for name in av.vectors:
  ax.plot(av.get_best(name, relu_th=0.8), label='v:'+name, alpha=0.5)
ax.plot([0.5]*av.size, label='0.5', alpha=0.2)
plt.legend(loc='upper right')



GLOBALS__['renderer'].render_color_text(TOKENS, av.get_by_name('_custom'), _range=(0,1) )
# TOKENS

# Fuzzy Matcher

In [0]:
from fuzzy_matcher import AttentionVectors
from ml_tools import sum_probabilities, subtract_probability, relu, momentum_p
import numpy as np
from ml_tools import FixedVector, FixedVectors,Tokens


def group_indices(_indices, cut_threshold=2):
  indices = sorted(_indices)

  _start = indices[0]
  _prev = indices[0]
  slices = []
  
  _n=len(indices)

  for i_i in range(_n):
    i = indices[i_i]

    if i - _prev > cut_threshold:
      slices.append(slice(_start, _prev + 1))
      _start = i
      
    if  i_i == _n - 1:
      slices.append(slice(_start, i + 1))
      
    _prev = i
  
  return slices
  
  
  
# group_indices([2,8])


class FuzzyMatcher:
  def __init__(self, av: AttentionVectors):
    self.av = av
    self.constraints = []
    self.incusions = []
    
    

  def _add(self, name, max_tokens, multiplyer, to_left):
    assert self.av.has(name), f'there is no vector named {name}'
    self.constraints.append((name, max_tokens, multiplyer, to_left))

  def _include(self, name, include):
    assert self.av.has(name), f'there is no vector named {name}'
    self.incusions.append((name, include))

  def after(self, name, max_tokens: int, multiplyer=1) -> 'FuzzyMatcher':
    self._add(name, max_tokens, multiplyer,  False)
    return self

  def before(self, name, max_tokens: int, multiplyer=1) -> 'FuzzyMatcher':
    self._add(name, max_tokens, multiplyer,  True)
    return self

  def excluding(self, name) -> 'FuzzyMatcher':
    self._include(name, False)
    return self

  def including(self, name) -> 'FuzzyMatcher':
    self._include(name, True)
    return self
  
  def _get_by_prefix(self, prefix ) -> FixedVectors:
    for name in self.av.get_names_by_pattern(prefix):
      yield self.av.get_by_name(name )

  def _get(self, name, p_threshold=0.5):
    _relu = 0.8
    if name[-1] == '*':
      vectors = self._get_by_prefix(name[:-1] )
      v = sum_probabilities( [relu(v, p_threshold) for v in vectors])
    else:
      v = self.av.get_by_name (name)
      v = relu(v, p_threshold) 
      
    return v
  
  def extract_name(self, attention: FixedVector, tokens: Tokens, cut_threshold = 2) -> [slice]:
    best_indices = []
    #     best_indices = sorted(np.argsort(attention)[::-1][:20])

    for i in np.argsort(attention)[::-1][:20]:
      if attention[i] > 0.001:
        best_indices.append(i)
    best_indices = sorted(best_indices)

    if len(best_indices) == 0:
      return []

    
    slices = group_indices(best_indices, cut_threshold)

    _dict = {}
    for s in slices:
      key = '__'.join(tokens[s])
      _sum = np.max(attention[s])
      _dict[key] = (_sum, s)

    _arr = [x[1] for x in sorted(_dict.values(), key=lambda item: -item[0])]

    return _arr

  def compile(self, p_threshold=0.33, bias=-0.33) -> np.ndarray:
 
    # regions
    _momented = []
    _momented.append(np.zeros(self.av.size))
    for name, max_tokens, multiplyer, to_left in self.constraints:
      v = self._get(name, p_threshold) + bias
      
      v = relu(v, p_threshold)  # TODO: remove? do not ReLu here?
      
      momented = momentum_p(v, half_decay=max_tokens, left=to_left)
      momented -= v
      
      momented *= multiplyer
#       momented = relu(momented, 0.5)
      _momented.append(momented)

    m_result = sum_probabilities(_momented)

    # incusions
    _included = []
    _included.append(np.zeros(self.av.size))
    for name, include in self.incusions:
      v = self._get(name, p_threshold) + bias
      if include:
        _included.append(relu(v, p_threshold))
    i_result = sum_probabilities(_included)

    # exclusions
    _excluded = []
    _excluded.append(np.zeros(self.av.size))
    for name, include in self.incusions:
      v = self._get(name, p_threshold)+bias
      if not include:
        _excluded.append(relu(v, p_threshold))
    e_result = sum_probabilities(_excluded)

    m_result = sum_probabilities([m_result, i_result])
    m_result = subtract_probability(m_result, e_result)

    #     result = relu(result, 0.001)
    return m_result
  
  
  
FuzzyMatcher2=FuzzyMatcher
 

### Make patterns

In [0]:
 
from patterns import AbstractPatternFactory, FuzzyPattern

from text_tools import acronym

ORG_TYPES = {
  'org_unknown': 'undefined',
  'org_ao':   'Акционерное общество',
  'org_zao':  'Закрытое акционерное общество',
  'org_oao':  'Открытое акционерное общество',
  'org_ooo':  'Общество с ограниченной ответственностью',
  'org_nc':   'Некоммерческая организация',
  'org_fpsi': 'Фонд поддержки социальных инициатив',
  'org_any_1': 'Учреждение',
  'org_any_2': 'Организация',
  'org_gau': 'Государственное автономное учреждение',
  'org_roo': 'Общественная организация'
   
}




class CPF(AbstractPatternFactory):
  def __init__(self):
    AbstractPatternFactory.__init__(self, None)
    self._build_ner_patterns()

  def _build_ner_patterns(self):
    def cp(name, tuples):
      assert tuples[1] != '', name
      return self.create_pattern(name, tuples)

    for o_type in ORG_TYPES.keys():
      if o_type != 'org_unknown':
        n = ORG_TYPES[o_type]
        cp(o_type, ('', n, ' «'))
        
        an = acronym(n)
        if(len(an)>1):
          cp(o_type+'_acronym', ('', an, ' «'))

    
    cp('_named_1', ( ',', 'именуемое в дальнейшем', '«'))
    cp('_named_2', ( ',', 'именуемое далее', '«'))
    cp('_named_4', ( ',', 'именуемое далее', '–'))
    cp('_named_5', ( ',', 'именуемое в дальнейшем', '–'))
    
    cp('_place_1', ( '', 'место нахождения', ''))
    cp('_place_2', ( '', 'город', 'Госква'))
    
   
#     cp('_named_3', ( ',', 'далее', '– «'))
    
    
    cp('_made_a_deal', ( '', 'заключили', 'настоящий договор о нижеследующем'))
    
    cp('_acting_on_1', ( ',', 'действующего на основании', 'устава'))
    cp('_acting_on_2', ( ',', 'действующего на основании', 'доверенности'))
 
    
    cp('_deal_side_1', ( ',', 'с одной стороны', ''))
    cp('_deal_side_2', ( ',', 'с другой стороны', ''))
    

    cp('_in_face_1', ( '', 'в лице', ''))
    cp('_in_face_2', ( '', 'в лице', 'Генерального директора'))
    
    
    
    px = ', именуемое в дальнейшем'
    cp('_alias_out.1', ( px, 'Покупатель', ''))
    cp('_alias_out.2', ( px, 'Заемщик', ''))
    cp('_alias_out.3', ( px, 'Арендатор', ''))
    cp('_alias_out.4', ( px, 'Жертвователь', ''))
    cp('_alias_out.5', ( px, 'Благотворитель', ''))
    cp('_alias_out.5', ( px, 'Заказчик', ''))
    
    cp('_alias_in.1', ( px, 'Исполнитель', ''))
    cp('_alias_in.2', ( px, 'Продавец', ''))    
    

# ---
contract_pattern_factory = CPF()
 

for o_type in ORG_TYPES.keys():
  print(acronym(ORG_TYPES[o_type]),'\t', ORG_TYPES[o_type])

In [0]:
for p in contract_pattern_factory.patterns:
  print(p.name, f'\t"{p.prefix_pattern_suffix_tuple[1]}"')

In [0]:
%%time
av = PM.find_patterns(text_tokens=TOKENS, patterns=contract_pattern_factory.patterns)



# Batch

### load files from gdrive

In [0]:
# import gspread
from google.colab import auth
from google.colab import drive
# from oauth2client.client import GoogleCredentials

auth.authenticate_user()
# google_spread = gspread.authorize(GoogleCredentials.get_application_default())

drive.mount('/content/gdrive', force_remount=True)


import glob
def read_documents(filename_prefix):
  texts = {}
  for file in glob.glob(filename_prefix+"*.doc"):
    try:
      text = read_doc(file)
      texts[file] = text
      print("good:", file)
    except:
      print('WRONG *.doc FILE!!', file)

  for file in glob.glob(filename_prefix+"*.docx"):
    try:
      text = read_doc(file)
      texts[file] = text
      print("good:", file)
    except:
      print('WRONG *.docx FILE!!', file)
      
  return texts

contracts_filename_prefix='/content/gdrive/My Drive/GazpromOil/Contracts/'
contracts = read_documents(contracts_filename_prefix)

In [0]:
BAD_DOCS="""
ДКП Половинной НБ и АЗС.docx
2. Договор по благ-ти Радуга.docx
Проект договора Пур адм.docx
Договор аренды.docx
5. договор Залога.docx
4. Договор Собственник.docx
Договор выполнения работ_Петров.docx
1.1. Договор пожертвования.docx
Договор 4.docx
Договор 5.docx
Договор1.docx
Проект договора Пур адм - благотв.docx
Договор 6.docx
Договор 3.docx
ДОГОВОР_Детский дом.docx
ГПН 3 Договор пожертвования 21.02.2019.doc
"""

BAD_DOCS = [ '/content/gdrive/My Drive/GazpromOil/Contracts/'+x.strip() for x in BAD_DOCS.split('\n') if x.strip()!='' ]
for i in  range(len(BAD_DOCS)):
  print(i, BAD_DOCS[i])

### Debug, see your data

#### Match aliases 

In [0]:
txt = contracts[BAD_DOCS[3]] 


from contract_parser import make_company_names_fm, match_contractor_aliases
from text_tools import hot_punkt, hot_quotes

 

In [0]:



# def match_contractor_aliases(_av):
#   alias_matcher = FuzzyMatcher2(_av)
  
# #   alias_matcher.including('_named_*')

#   alias_matcher.after('_named_*', 10, 1)
#   alias_matcher.before('_in_face*', 5, 0.8) 
#   alias_matcher.excluding('_named_*')
#   alias_matcher.excluding('_punkt')
#   alias_matcher.including('_alias_*')
  
  
#   alias_matcher.after('_quotes_open', 5, 0.5) .before('_quotes_closing', 5, 0.5) 
#   alias_matcher .before('_deal_side_*', 5, 0.5)
  
 
#   alias_attention = alias_matcher.compile(p_threshold=0.4)
# #   aliaces_indices = sorted(np.argsort(alias_attention)[::-1][:2])
  
#   alias_attention=relu(alias_attention, 0.9)
#   return alias_attention


# _av, token, alias_attentions = process_contract_aliases(txt)



# alias_attention = match_contractor_aliases(_av)
# _av.add('alias_attention', alias_attention)


# GLOBALS__['renderer'].render_color_text(tokens, _av.get_by_name('alias_attention'),  _range=[0, 1])


#### Find NEs

In [0]:

txt = contracts[BAD_DOCS[1]]

def process_contract(txt):

  tokens = tokenize_text(normalize_text(txt, replacements_regex))
  tokens = tokens[:200]
  
#   print(tokens)
  _av = PM.find_patterns(text_tokens=tokens, patterns=contract_pattern_factory.patterns)
  _quotes_open, _quotes_closing = hot_quotes(tokens)
  _av.add('_quotes_open', _quotes_open * 1)
  _av.add('_quotes_closing', _quotes_closing * 1)
  _av.add('_punkt', hot_punkt(tokens))
  
  
  alias_attention = match_contractor_aliases(_av)
  _av.add('alias_attention', relu(alias_attention, 0.8))
  
  
  return _av, tokens
  
  
  

# txt = uploaded 
_av, tokens = process_contract(txt)



In [0]:



_fm, attention, alias_attention = make_company_names_fm(_av, relu_th=0.9)
# GLOBALS__['renderer'].render_color_text(tokens, alias_attention,  _range=[0, 1])
# print()

_mx = np.max(attention) * 0.95
print(_mx)
attention = relu(attention, max(0.85, _mx))
slices = _fm.extract_name(attention, tokens)
for s in slices[:2]:
  GLOBALS__['renderer'].render_color_text(tokens[s], attention[s], _range=(0,1))
print("-"*100)

GLOBALS__['renderer'].render_color_text(tokens, attention,  _range=[0, 1])
GLOBALS__['renderer'].render_color_text(tokens, _av.get_by_name('alias_attention'),  _range=[0, 1])





In [0]:
#All

## Batch: all

In [0]:
import numpy as np

# from fuzzy_matcher import FuzzyMatcher
from ml_tools import relu, cut_above
from text_normalize import replacements_regex, normalize_text
from text_tools import hot_quotes, my_punctuation, Tokens, tokenize_text, untokenize


 

def max_relu(x, a, k):
  _mx = np.max(x) * k
  print(_mx)
  return relu(x, max(a, _mx))

fc = 0
export_string_all = ''
for fn in contracts:#BAD_DOCS:# 
  fc += 1
  print()
  print(fc, '=' * 100)
  print(fc, fn)
  txt = contracts[fn]
  tokens = tokenize_text(normalize_text(txt, replacements_regex))
  tokens = tokens[:300]  # TODO: trimming just for the batch to speed things up

  _av = PM.find_patterns(text_tokens=tokens, patterns=contract_pattern_factory.patterns)
  _quotes_open, _quotes_closing = hot_quotes(tokens)
  _av.add('_quotes_open', _quotes_open * 1)
  _av.add('_quotes_closing', _quotes_closing * 1)
  _av.add('_punkt', hot_punkt(tokens))
 

  _fm, attention, alias_attention = make_company_names_fm(_av, relu_th=0.85)
  
#   attention = max_relu(attention, 0.85, 0.97)
#   alias_attention = max_relu(alias_attention, 0.85, 0.9)
  
  
  _as = sorted(_fm.extract_name(alias_attention, tokens)[0:2], key=lambda item: item.start)
  print("Aliases:")
  for s in _as:
    GLOBALS__['renderer'].render_color_text(tokens[s], alias_attention[s], _range=(0,1))
    
#   export_string += f'{fn.split("/")[-1]}\t{tokens[i1]}\t{tokens[i2]}\t{alias_attention[i1]}\t{alias_attention[i2]}\n'
  
  
  print("NEs:")
  _nm = sorted(_fm.extract_name(attention, tokens)[0:2], key=lambda item: item.start)
  for s in _nm:
    GLOBALS__['renderer'].render_color_text(tokens[s], attention[s], _range=(0,1))

  export_string=''
  export_string += f'{fn.split("/")[-1]}\t'
 
  if len(_as)==2:
    export_string += f'{untokenize(tokens[_as[0]])}\t{untokenize(tokens[_as[1]])}\t'
  else:
    export_string += '\t\t'
    
  if len(_nm)==2  :
    export_string += f'{untokenize(tokens[_nm[0]])}\t{untokenize(tokens[_nm[1]])}\t'
  else:
    export_string += '\t\t'
    
    
  export_string +='\n'
  
  print(export_string)
  export_string_all += export_string
#   # --------------------------------QUOTES attention-
#   _quotes_open, _quotes_closing = hot_quotes(tokens)
#   _av.add('_quotes_open', _quotes_open * 1)
#   _av.add('_quotes_closing', _quotes_closing * 1)
  
#   _av.add('hot_punkt', hot_punkt(tokens))
#   # ----------------------------END OF----QUOTES attention-

  
#   aliaces_indices, alias_attention = match_contractor_aliases(_av)
#   _av.add('alias_attention', alias_attention)
    
#   _fm = FuzzyMatcher2(_av)
  
#   _fm.after('org_*',10).excluding('org_*')
#   _fm.before('_quotes_closing', 10).after('_quotes_open', 10)
  
 
#   _fm.after('_deal_side_1', 30)  
#   _fm.before('_named_*', 40) 
#   _fm.before('_in_face', 10)\
#     .before('_acting_on', 40) \
#     .before('_deal_side_*', 100)\
#     .before('_made_a_deal', 200)\
#     .before('_in_face', 10)\
  
#   _fm.excluding('alias_attention').excluding('hot_punkt') 
  
 
    
# #   alias_attention = relu(alias_attention, min( )  )

#   _sel_t = [tokens[aliaces_indices[0]], '---', tokens[aliaces_indices[1]]]
#   _sel_v = np.array([alias_attention[aliaces_indices[0]], 0, alias_attention[aliaces_indices[1]]])

# #   GLOBALS__['renderer'].render_color_text(_sel_t, _sel_v, _range=[0, 1])

#   i1 = aliaces_indices[0]
#   i2 = aliaces_indices[1]

#   slices = _fm.extract_name(attention, tokens)
#   export_string += f'{fn.split("/")[-1]}\t{tokens[i1]}\t{tokens[i2]}\t{alias_attention[i1]}\t{alias_attention[i2]}\n'

#   alias_attention_sl = slice(i1 - 10, i2 + 10)
# #   GLOBALS__['renderer'].render_color_text(tokens[alias_attention_sl], alias_attention[alias_attention_sl], _range=(0,1))

# #   attention = _fm.compile(bias=-0.6)
# #   attention = relu(attention, 0.9 * np.max(attention))
  
#   best_indices = np.argsort(alias_attention)[::-1][:10]
#   best_index = np.min(best_indices)
#   last_index = np.max(best_indices)
    
#   print('NEs:')
  
#   best_indices = np.argsort(attention)[::-1][:10]
#   print("TOP TEN:")
#   for bi in best_indices:
#     GLOBALS__['renderer'].render_color_text(tokens[bi:bi+1], attention[bi:bi+1], _range=(0,1))
#   print("-------------------------:TOP TEN")
  
#   slices = _fm.extract_name(attention, tokens)
#   for s in slices:
#     GLOBALS__['renderer'].render_color_text(tokens[s], attention[s], _range=(0,1))

#   print('quote:')
# #   last_index = min(last_index, best_index+100)
# # #   print('BI:', best_indices)
#   _slice = slice(best_index, last_index+2)
#   GLOBALS__['renderer'].render_color_text(tokens[_slice], attention[_slice], _range=(0,1))


print(export_string_all)
